In [1]:
1+1

2

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# from scipy.integrate import odeint
# import sympy as sp

In [6]:
# Define the function of the dynamics
def cosine_dynamics(X, t, constant, alpha, sigma, B):
    x,y = X # x = angle = theta, y = dxdt = angular velocity = omega, dydt = angular acceleration

    conditions = [(x > (B - sigma)) & (x < (B + sigma)),
              (x <= (B - sigma)) | (x >= (B + sigma))]

    functions = [lambda x: 1 + np.cos(2 * np.pi * (x - B) / (2 * sigma)), lambda x: 0]

    dxdt = y
    dydt = constant * (x - np.sign(B) * alpha * np.piecewise(x, conditions, functions)) # minuses the cosine
    return [dxdt, dydt]

In [16]:
import numpy as np
from scipy.integrate import odeint
from scipy.optimize import minimize
import sympy as sp

# Constants
mass, height, setpoint_baseline, sd_baseline = 80, 1.68, 1.0292359, 0.4495318
scaling = 0.971
L = 1.09
g = 9.81
m = mass
H = height
I = 0.35 * m * H**2  # Distributed Inertia
constant = scaling * m * g * L / I

# Baselines in degrees
cos_start_deg = setpoint_baseline + 1 * sd_baseline
cos_acc0_deg = setpoint_baseline + 3 * sd_baseline

# Convert degrees to radians
cos_start = np.deg2rad(cos_start_deg)
cos_acc0 = np.deg2rad(cos_acc0_deg)

# Time range for simulation
t = np.linspace(0, 10, 1000)  # Adjust the time range as needed

# Define the function for dydt
def dydt(x, alpha, sigma, B):
    conditions = [(x > (B - sigma)) & (x < (B + sigma)),
                  (x <= (B - sigma)) | (x >= (B + sigma))]
    functions = [lambda X: (1 + np.cos(2 * np.pi * (X - B) / (2 * sigma))), lambda X: 0]
    dydt = scaling * m * g * L / I * (x - np.sign(B) * alpha * np.piecewise(x, conditions, functions))
    return dydt

# Objective function to minimize
def objective(params):
    alpha, sigma = params
    B = sigma + cos_start  # Criterion 1: B = sigma + cos_start
    # Initial condition
    y0 = cos_start
    # Integrate the ODE
    sol = odeint(dydt, y0, t, args=(alpha, sigma, B))
    x = sol[:, 0]
    # Criterion 2: dydt crosses zero at x = cos_acc0
    # Find time when x crosses cos_acc0
    idx = np.argmin(np.abs(x - cos_acc0))
    dydt_at_cos_acc0 = dydt(x[idx], t[idx], alpha, sigma, B)
    # Criterion 3: dydt = -0.05 when x = B
    dydt_at_B = dydt(B, 0, alpha, sigma, B) + 0.05
    # Compute the objective function
    obj = dydt_at_cos_acc0**2 + dydt_at_B**2
    # Penalize negative sigma and alpha
    if sigma <= 0 or alpha <= 0:
        obj += 1e6
    return obj

# Initial guesses for alpha and sigma
initial_guess = [1, 1]

# Bounds for alpha and sigma (positive values)
bounds = [(1e-6, None), (1e-6, None)]

# Optimize alpha and sigma
result = minimize(objective, initial_guess, bounds=bounds, method='L-BFGS-B')

# Extract optimized parameters
alpha, sigma = result.x
B = sigma + cos_start

# Print the results
print(f"Alpha: {alpha}")
print(f"Sigma: {sigma}")
print(f"B: {B}")

# Print the results
print(f"strength: {alpha}")
print(f"width: {np.rad2deg(sigma)}")
print(f"location: {np.rad2deg(B)}")

KeyboardInterrupt: 

In [17]:
import numpy as np
from scipy.optimize import fsolve
import sympy as sp


mass, height, setpoint_baseline, sd_baseline = 80, 1.68, 1.0292359, 0.4495318
#Constants
scaling = 0.971
L = 1.09
g = 9.81
m = mass #change
H = height #change
I = 0.35*m*H**2     # Distributed Inertia
b = 0
maxAngle = 65
constant = scaling * m * g * L / I

## baselines
setpoint_baseline = setpoint_baseline #change
sd_baseline = sd_baseline #change
cos_start = np.deg2rad(setpoint_baseline + 1 * sd_baseline) 
cos_acc0 = np.deg2rad(setpoint_baseline + 3 * sd_baseline) 

# Define the function for dydt
def dydt_function(x, alpha, sigma, B):
    conditions = [(x > (B - sigma)) & (x < (B + sigma)),
                  (x <= (B - sigma)) | (x >= (B + sigma))]
    functions = [lambda X: (1 + np.cos(2 * np.pi * (X - B) / (2 * sigma))), lambda X: 0]
    dydt = scaling * m * g * L / I * (x - np.sign(B) * alpha * np.piecewise(x, conditions, functions))
    return dydt

# get sigma and B
sigma = sp.Symbol('sigma')
# X = cos_acc0
B = sigma + cos_start
delta = -0.05 / constant
alpha = (B - delta) / 2
equation = constant * (cos_acc0 - alpha * (1 + sp.cos((2 * sp.pi * (cos_acc0 - B)) / (2 * sigma)))) 
solutions = sp.solve(equation, sigma)
sigma_out = float(solutions[1]) ## stability width
# Print the results
print(f"strength: {((sigma_out + cos_start) - delta) / 2}")
print(f"width: {np.rad2deg(sigma_out)}")
print(f"location: {np.rad2deg(sigma_out + cos_start)}")

# # Compute delta from Equation 3
# delta = -0.05 / constant

# # Define the function to solve for B
# def f_B(x, B):
#     alpha = (B - delta) / 2  # From rearranged Equation 3
#     sigma = B - cos_start    # From Equation 1
#     # Avoid division by zero or negative sigma
#     if sigma <= 0:
#         return np.inf
#     conditions = [(x > (B - sigma)) & (x < (B + sigma)),
#                   (x <= (B - sigma)) | (x >= (B + sigma))]
#     functions = [lambda X: (1 + np.cos(2 * np.pi * (X - B) / (2 * sigma))), lambda X: 0]
#     dydt = scaling * m * g * L / I * (x - np.sign(B) * alpha * np.piecewise(x, conditions, functions))
#     return dydt
    # argument = (cos_acc0 - B) * np.pi / sigma
    # # Ensure the argument is within the valid range for cosine
    # cos_term = np.cos(argument)
    # # Equation 2 rearranged
    # lhs = cos_acc0
    # rhs = alpha * (1 + cos_term)
    # return lhs - rhs

# Initial guess for B (must be greater than cos_start to ensure positive sigma)
initial_guess = cos_start

# Solve for B using fsolve
B_solution = fsolve(f_B, initial_guess)
print(B_solution)
# Extract B and compute alpha and sigma
B = B_solution[0]
alpha = (B - delta) / 2
sigma = B - cos_start

# Print the results
print(f"Alpha: {alpha}")
print(f"Sigma: {sigma}")
print(f"B: {B}")

# Print the results
print(f"strength: {alpha}")
print(f"width: {np.rad2deg(sigma)}")
print(f"location: {np.rad2deg(B)}")

RecursionError: maximum recursion depth exceeded

In [11]:
#function to get max velocity for a specific alpha
def get_max_velocity(alpha, cos_start, cos_acc0, constant):
    # get sigma and B
    sigma = sp.Symbol('sigma')
    # X = cos_acc0
    B = sigma + cos_start
    print(alpha, cos_start, cos_acc0, constant)
    equation = constant * (cos_acc0 - alpha * (1 + sp.cos((2 * sp.pi * (cos_acc0 - B)) / (2 * sigma)))) 
    solutions = sp.solve(equation, sigma)
    print(solutions)
    try: 
        sigma = float(solutions[1]) ## stability width
    except TypeError:
        return None,None,None
    B = sigma + cos_start
    # print(B, sigma)
    max_velocity = None
    for i in np.arange(cos_start, cos_acc0, 0.0001):
        # print(i)
        X = [i,0]
        t = np.linspace(0, 5, 100000)
        sol = odeint(cosine_dynamics, X, t, args=(constant, alpha, sigma, B)) #integrates dxdt and dydt to get theta (angle) and omega (velocity)
        theta = np.rad2deg(sol[:,0])
        omega = np.rad2deg(sol[:,1])
        # print(theta)
        # print(np.rad2deg(cos_acc0))
        if (any(x < 0 for x in omega)):
            max_velocity = max(omega)
            break
    if isinstance(max_velocity, (int, float)):
        return max_velocity, B, sigma
    else: return None,None,None

In [4]:
def get_parameters(mass, height, setpoint_baseline, sd_baseline, maxV):
    #Constants
    scaling = 0.971
    L = 1.09
    g = 9.81
    m = mass #change
    H = height #change
    I = 0.35*m*H**2     # Distributed Inertia
    b = 0
    maxAngle = 65
    constant = scaling * m * g * L / I

    ## baselines
    setpoint_baseline = setpoint_baseline #change
    sd_baseline = sd_baseline #change
    cos_start = np.deg2rad(setpoint_baseline + 1 * sd_baseline) 
    cos_acc0 = np.deg2rad(setpoint_baseline + 3 * sd_baseline) 
    # print(cos_start, cos_acc0, constant)

    alpha_out = 0
    velocity_out = 0
    for alpha in np.arange(0.01, 0.04, 0.0001):
        max_velocity, B, sigma = get_max_velocity(alpha, cos_start, cos_acc0, constant)
        if isinstance(max_velocity, (int, float)):
            if max_velocity < maxV:
                alpha_out = alpha
                B_out = B
                sigma_out = sigma
                velocity_out = max_velocity
            else:
                print("alpha: ", alpha_out)
                print("peak: ", np.rad2deg(B_out))
                print("width: ", np.rad2deg(sigma_out))
                print(velocity_out)
                break
        else:
            continue



In [ ]:
# get_parameters(weight (kg), height (m), setpoint, sd everything, max V)

In [13]:
get_parameters(51, 1.62, 0.823786215236112, 0.705889947960325, 2)

0.01 0.026697885537051935 0.05133809303433014 11.303596903782088
[0.0170436226789449 - 0.01137863459701*I, 0.0170436226789449 + 0.01137863459701*I]
0.0101 0.026697885537051935 0.05133809303433014 11.303596903782088
[0.0171075823787027 - 0.0113518723012512*I, 0.0171075823787027 + 0.0113518723012512*I]
0.010199999999999999 0.026697885537051935 0.05133809303433014 11.303596903782088
[0.0171712020327227 - 0.0113248312047197*I, 0.0171712020327227 + 0.0113248312047197*I]
0.010299999999999998 0.026697885537051935 0.05133809303433014 11.303596903782088
[0.0172344868761308 - 0.0112975127728831*I, 0.0172344868761308 + 0.0112975127728831*I]
0.010399999999999998 0.026697885537051935 0.05133809303433014 11.303596903782088
[0.0172974420126204 - 0.0112699183751764*I, 0.0172974420126204 + 0.0112699183751764*I]
0.010499999999999997 0.026697885537051935 0.05133809303433014 11.303596903782088
[0.0173600724189952 - 0.0112420492873435*I, 0.0173600724189952 + 0.0112420492873435*I]
0.010599999999999997 0.026

In [ ]:
get_parameters(51, 1.62, 0.823786215236112, 0.705889947960325, 2)

In [5]:
## pilot3
get_parameters(51, 1.62, 0.823786215236112, 0.705889947960325, 1.475)

alpha:  0.03149999999999987
peak:  3.499083337841862
width:  1.969407174645425
1.4545005021697046


# P5

In [5]:
get_parameters(80, 1.68, 1.0292359, 0.4495318, 1.620) # sd_everything + personal maxV

alpha:  0.0267999999999999
peak:  2.7916185692294273
width:  1.3128508692294276
1.6174125807810136


## P4

In [44]:
get_parameters(69, 1.72, 0.404, 0.266, 2.011) #avg_sd + global maxV
# find it? no
# abrupt? no , can feel the start of the stability
# able to hold? yes

alpha:  0.01809999999999995
peak:  1.6357906014216916
width:  0.9657906014216916
2.0071240029304507


In [45]:
get_parameters(69, 1.72, 0.404, 0.584, 2.011) # sd_everything + global maxV
# find it? no
# abrupt? no, they like it can't tell until they are there (a bit abrupt going through it)
# able to hold? yes

alpha:  0.026899999999999896
peak:  2.840223034490924
width:  1.8522230344909238
2.0012591127169026


In [46]:
get_parameters(69, 1.72, 0.404, 0.266, 1.440) # avg_sd + personal maxV
# find it? yes
# abrupt? very smooth, going past it feels normal, the smoothest so far 
# able to hold? yes-ish, if fulling relaxed, will fall through it, still helping and easier to stand

alpha:  0.015899999999999963
peak:  1.551539410551427
width:  0.881539410551427
1.428141228638181


In [47]:
get_parameters(69, 1.72, 0.404, 0.584, 1.440) # sd_everything + personal maxV
# find it? no, stability is very far forward
# abrupt? smooth, hard to tell where it starts, can feel being attracted to it
# able to hold? quite good, holds pretty well

alpha:  0.02489999999999991
peak:  2.729191334794012
width:  1.741191334794012
1.4237698079899825


## P2

In [48]:
get_parameters(51, 1.63, 0.110, 0.363, 2.011) #avg_sd + global maxV
# find it? Yes
# abrupt? No
# able to hold? almost no
# ossilation? Yes 

alpha:  0.017999999999999954
peak:  1.7882588672581103
width:  1.3152588672581105
2.0064247672522217


In [49]:
get_parameters(51, 1.63, 0.110, 0.423, 2.011) # sd_everything + global maxV
# find it?  Yes
# abrupt? no 
# able to hold? Yes， stronger
# ossilation? small oscillation， better than before， feels close 0

alpha:  0.01969999999999994
peak:  2.0141688961269164
width:  1.481168896126916
1.9915344258543102


In [50]:
get_parameters(51, 1.63, 0.110, 0.363, 2.295) # avg_sd + personal maxV
# find it? Yes
# abrupt? no
# able to hold? ye s 
# ossilation? Bigger compared previous， better than first

alpha:  0.018899999999999945
peak:  1.8320527720724302
width:  1.3590527720724301
2.2679513339529285


In [51]:
get_parameters(51, 1.63, 0.110, 0.423, 2.295) # sd_everything + personal maxV
# find it? yes
# abrupt? no 
# able to hold? stronger hold than  test 2， very strong
# ossilation? same oscillations test 2

alpha:  0.020699999999999934
peak:  2.0655437459883075
width:  1.5325437459883073
2.2864209924947443


## P3

In [54]:
get_parameters(77, 1.65, 0.818, 0.262, 2.011) #avg_sd + global maxV #location, width, strength
# find it? yes
# abrupt? yes, very smooth
# able to hold? slowly - yes, fast - speed bump
# ossilation? yes

alpha:  0.02129999999999993
peak:  1.9507537056665025
width:  0.8707537056665025
1.981898117467969


In [55]:
get_parameters(77, 1.65, 0.818, 0.367, 2.011) # sd_everything + global maxV
# find it? yes
# abrupt? no going into it
# able to hold? yes, but when balancing, don't trust it
# ossilation? yes, unphysiological, has a very characteristic frequency, unlike the stochastic motion

## this one feels the same as the previous one, put in damping

alpha:  0.023899999999999914
peak:  2.347371471606372
width:  1.1623714716063718
1.9804651804147335


In [56]:
get_parameters(77, 1.65, 0.818, 0.262, 2.605) # avg_sd + personal maxV
# find it? feels further forward
# abrupt? fine
# able to hold? feels stronger
# ossilation? has a higher frequecy vibration

alpha:  0.02359999999999992
peak:  2.016336801177448
width:  0.9363368011774478
2.5926854974479974


In [57]:
get_parameters(77, 1.65, 0.818, 0.367, 2.605) # sd_everything + personal maxV
# find it? yes, feels closer than trial 3
# abrupt? no
# able to hold? yes
# ossilation? does not have high ossilation like trial 3

alpha:  0.026199999999999904
peak:  2.429513382463942
width:  1.2445133824639416
2.5973909393024246


# P1

In [5]:
get_parameters(90, 1.73, 0.610, 0.409, 2.011) #avg_sd + global maxV #location, width, strength
# find it? no
# abrupt? no
# able to hold? yes
# ossilation? yes

alpha:  0.02359999999999992
peak:  2.345355512931836
width:  1.3263555129318363
1.984744254833987


In [6]:
get_parameters(90, 1.73, 0.610, 0.565, 2.011) # sd_everything + global maxV
# find it? yes
# abrupt? no, but the pulling from the stability point really distrcted the normal standing balance (too close to setpoint) pulling is stronger than test1
# able to hold? yes, still having some forces in the muscle, good to have longer trials
# ossilation? yes


alpha:  0.027999999999999893
peak:  2.9299822127863377
width:  1.754982212786338
1.9997153866363997


In [7]:
get_parameters(90, 1.73, 0.610, 0.409, 2.095) #avg_sd + global maxV #location, width, strength
# find it? yes
# abrupt? more pointy, no pull, more curiosity to find it
# able to hold? yes, if turn off all the muscles will fall, feel easier to stand
# ossilation? yes 

## Amin's favorate. he has to actively explore to find it

alpha:  0.023999999999999917
peak:  2.3620933509601443
width:  1.343093350960144
2.092789161624924


In [8]:
get_parameters(90, 1.73, 0.610, 0.565, 2.095) # sd_everything + global maxV
# find it? yes
# abrupt? yes, also feels the pull when stability starts
# able to hold? yes, stronger than previous ones, when turn off all the muscles, will fall
# ossilation? yes

alpha:  0.028299999999999888
peak:  2.9447355375140516
width:  1.7697355375140518
2.079943541120101
